In [4]:
%%capture
%run 01_transaction.ipynb

In [5]:
from eth_account.messages import encode_defunct as encode_msg
import web3 as w3; w3 = w3.Account

Create a public key `pub_key`, private key `priv_key` pair. The public key is used as an address and the private key is used for signing txs. The `priv_key` is basically your password.

In [6]:
def keys():
    acc = w3.create()
    return acc.privateKey.hex(), acc.address

priv_key,pub_key = keys()
ph(priv_key), ph(pub_key)

('📬 0x861e2916ca...dbc', '👦 0x001f7A7c7C...Be5')

## Account

Holds a key pair and is used for signing txs. The `nonce` keeps track of the number of txs made with this account.

In [7]:
class Account:
    def __init__(self): 
        self.priv, self.pub = keys()
        self.nonce = 0
        
    def sign(self, tx):
        self.nonce += 1
        m = encode_msg(bytes(tx))
        sig = w3.sign_message(m, self.priv)
        tx.sig = sig
        return tx
            
    def signed_tx(self, to, value, fee):
        tx = TX(self.pub, to.pub, value, fee, self.nonce)
        return self.sign(tx)
    
    def __str__(self): return ph(self.pub)

In [8]:
acc1 = Account(); print(acc1)
acc2 = Account(); print(acc2)

🔩 0xc37a13b440...D02
💎 0x283e55cBA6...B53


### Transaction Signing
Signing a tx generates a signature that only the private key can produce. This can be later validated very efficiently.

In [9]:
tx        = TX(acc1.pub, acc2.pub, 12, 0.2, acc1.nonce)
tx_signed = acc1.sign(tx)

The signature contains the tx hash `message_hash` and its `signature`. We don't need `r`, `v` and `s` as they are specific to Ethereum. For more infos: https://medium.com/@angellopozo/ethereum-signing-and-validating-13a2d7cb0ee3

In [10]:
tx_signed.sig

SignedMessage(messageHash=HexBytes('0x15e15d0168e23644fe9d3adab7956e3fd32cd372e28e4838b06ef5299f6acd94'), r=60479667066283719835134271062121644370405066253890898805939283033234703976651, s=12634778251723130648927824099007472111020145826627082767465589776775779283625, v=27, signature=HexBytes('0x85b6474abce51463b8d870eb7de0fc51092e6cac5c83162cd07b3d18d11508cb1bef07f409e3b8cfc2878354babfbb25b1c67e8254a5de7338796d89a91d72a91b'))

Accounts can generate a signed tx quickly with `signed_tx`.

In [11]:
tx = acc1.signed_tx(acc2, 7, 0.3); print(tx)

time:	Tue Mar 30 16:10:53 2021
from:	🔩 0xc37a13b440...D02
to:	💎 0x283e55cBA6...B53
value:	7 ether
fee:	0.3 ether
nonce:	1
hash:	💲 0x4c2e16d180...593
signed:	true


### Transaction Validation

Validates if the tx was signed by the sender.

In [12]:
def val_sig(tx):
    if not hasattr(tx, 'sig'): return False
    m = encode_msg(bytes(tx))
    return w3.recover_message(m, signature=tx.sig.signature) == tx.fr

In [13]:
assert val_sig(tx_signed)

If anything in the tx is changed, like increasing the value to send, the signature should become invalid.

In [14]:
tx_signed.value = 30
assert not val_sig(tx_signed)